In [21]:
! pip install nltk
! pip install sklearn

In [104]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [69]:
import sqlite3
import pandas as pd
import numpy as np

In [70]:
con = sqlite3.connect('C:\\Users\\nasty\\Project Python 2022\\karelia_database_project.db')  # подключение
cur = con.cursor()
from sqlite3 import connect
conn = connect(':memory:')

In [71]:
utterances_query = """
SELECT * FROM utterances
"""

In [72]:
df = pd.read_sql(utterances_query, con)
df

,utterance_id,speaker_id,speaker_role,interwiev_id,text,utterance_number
0,1,3,interviewer,1,"Вот, и, в общем, в Ведлозеро мы тоже заедем об...",1
1,2,1,informant,1,Да. там с Наташей лучше всего с Антоновой [кар...,2
2,3,3,interviewer,1,"Мхм. Да, мы знакомы даже с ней.",3
3,4,1,informant,1,"Ну, Зара такой хороший языковой активист в это...",4
4,5,3,interviewer,1,"Нет, а расскажите, что это такая за ассоциация?",5
...,...,...,...,...,...,...
3373,3374,52,informant,20,"С цепи срывают. Если бы не на цепи, она бы убе...",154
3374,3375,51,informant,20,Или из будки вытаскивают.,155
3375,3376,52,informant,20,"В последние годы на балкон медведи, тоже видел...",156
3376,3377,51,informant,20,"Мы когда строили дачу, у нас была собака, стор...",157


In [73]:
import re
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nasty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [74]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_ru = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
print(stopwords_ru)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [76]:
from pymorphy2 import MorphAnalyzer

In [77]:
#stopwords_ru = stopwords_ru.extend(['а', 'но', 'ой'])
#print(stopwords_ru)

In [78]:
morph = MorphAnalyzer()

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-+–""…“”]"
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    if doc.split() != None:
        for token in doc.split():
            if token and token not in stopwords_ru:
                token = token.strip()
                token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [79]:
doc = re.sub(patterns, ' ', df['text'][0])

In [80]:
stopwords_ru

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [81]:
for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]

In [82]:
df['text'][0]

'Вот, и, в общем, в Ведлозеро мы тоже заедем обязательно, но вот сегодня мы у вас.'

In [83]:
df

,utterance_id,speaker_id,speaker_role,interwiev_id,text,utterance_number
0,1,3,interviewer,1,"Вот, и, в общем, в Ведлозеро мы тоже заедем об...",1
1,2,1,informant,1,Да. там с Наташей лучше всего с Антоновой [кар...,2
2,3,3,interviewer,1,"Мхм. Да, мы знакомы даже с ней.",3
3,4,1,informant,1,"Ну, Зара такой хороший языковой активист в это...",4
4,5,3,interviewer,1,"Нет, а расскажите, что это такая за ассоциация?",5
...,...,...,...,...,...,...
3373,3374,52,informant,20,"С цепи срывают. Если бы не на цепи, она бы убе...",154
3374,3375,51,informant,20,Или из будки вытаскивают.,155
3375,3376,52,informant,20,"В последние годы на балкон медведи, тоже видел...",156
3376,3377,51,informant,20,"Мы когда строили дачу, у нас была собака, стор...",157


In [84]:
texts = df['text'].to_list()
bag = []
#topics = [row[6] for row in df]

# Process the texts to so they are ready for training
# But transform the list of words back to string format to feed it to sklearn
#texts = [" ".join(lemmatize(text)) for text in texts]

for text in texts:
    words = lemmatize(text)
    print(words)
    if words != None:
        li = ' '.join(words)
        bag.append(li)
    else:
        bag.append('')

['вот', 'и', 'в', 'общий', 'в', 'ведлозеро', 'мы', 'тоже', 'заехать', 'обязательно', 'но', 'вот', 'сегодня', 'мы', 'у', 'вас']
['да', 'там', 'с', 'наташа', 'лучше', 'всего', 'с', 'антонов', 'карельский', 'поэтесса', 'и', 'переводчица', 'об', 'этом', 'разговаривать']
['мхм', 'да', 'мы', 'знакомый', 'даже', 'с', 'ней']
['ну', 'зара', 'такой', 'хороший', 'языковой', 'активист', 'в', 'этом', 'план', 'а', 'с', 'ассоциация', 'эхо', 'вы', 'общаться']
['нет', 'а', 'рассказать', 'что', 'это', 'такой', 'за', 'ассоциация']
['это', 'ассоциация', 'этнокультурный', 'центр', 'карелия', 'там', 'просто', 'работать', 'два', 'специалист', 'который', 'очень', 'серьёзный', 'уровень']
['ой', 'а', 'поделиться', 'имя', 'их']
['да', 'алина', 'чубуров', 'и', 'алексей', 'цыкарев', 'цыкарев', 'он', 'вообще']
['а', 'конечно', 'конечно', 'знать', 'да', 'да']
['и', 'у', 'них', 'тоже', 'очень', 'много', 'как', 'бы', 'с', 'язык', 'но', 'они', 'отойти', 'так', 'немножко', 'от', 'этой', 'тема', 'но', 'лёша', 'понятно', 

['жизнь', 'это', 'жизнь', 'но', 'на', 'южный', 'карельский', 'язык']
['на', 'южный', 'на', 'ливвиковский']
['и', 'на', 'ливвиковский', 'и', 'на', 'людиковский', 'тоже', 'это', 'у', 'нас', 'есть', 'два', 'такой', 'активист', 'языковой', 'карелов', 'который', 'почему', 'то', 'продавить', 'что', 'в', 'пряжа', 'всё', 'таки', 'должный', 'быть', 'но', 'теперь', 'я', 'вам', 'мочь', 'сказать', 'вот', 'пройти', 'три', 'по', 'мой', 'год', 'или', 'четыре', 'как', 'они', 'название', 'убить', 'да', 'они', 'вообще', 'теперь', 'название', 'не', 'использовать', 'они', 'просто', 'этнокультурный', 'центр', 'ну', 'потому', 'что', 'оно', 'не', 'звучать', 'то', 'есть', 'это', 'как', 'раз', 'к', 'вопрос', 'про', 'сохранение', 'язык', 'и', 'всего', 'остальной', 'вопрос', 'ведь', 'не', 'только', 'в', 'том', 'чтобы', 'добиться', 'того', 'чтобы', 'называться', 'оно', 'именно', 'так', 'да', 'вопрос', 'маркетинг', 'уже', 'во', 'многое', 'потому', 'что', 'не', 'продаваться', 'а', 'продаваться', 'ну', 'в', 'хороший

['и', 'сейчас', 'проводить', 'это', 'прям', 'то', 'что', 'такой', 'тёплый', 'связанный', 'они', 'так', 'хорошо']
['я', 'забыть', 'как', 'называться', 'это', 'мероприятие']
['кабинет', 'оформлять', 'для', 'этот', 'дело', 'что', 'это', 'прям', 'очень', 'красиво', 'всякий', 'табличка', 'ставить']
['а', 'карельский', 'появиться', 'только', 'год', 'пять', 'назад', 'в', 'школа', 'в', 'программа']
['ну', 'да', 'это', 'с', 'преподаватель', 'связать']
['нет', 'он', 'был', 'и', 'долгий', 'но', 'на', 'сам', 'дело', 'всегда', 'вопрос', 'именно', 'в', 'преподавание']
['возможно', 'он', 'был', 'ранний']
['сложно', 'ведь', 'с', 'учитель', 'и', 'так', 'сложно', 'в', 'сельский', 'школа', 'не', 'важно', 'на', 'какой', 'язык', 'понимать', 'а', 'с', 'учитель', 'который', 'готовый', 'карельский', 'преподавать', 'ещё', 'сложный', 'а', 'почему', 'интересно', 'так', 'да', 'потому', 'что', 'письменность', 'карельский', 'в', 'общий', 'то', 'с', 'год', 'только', 'идти', 'то', 'есть', 'это', 'даже', 'под', 'вопро

['вот', 'у', 'карелов', 'это', 'есть', 'но', 'исключение', 'у', 'нас', 'это', 'эссойло', 'это', 'вообще', 'я', 'не', 'знать', 'это', 'отдельный', 'феномен', 'просто', 'потому', 'что', 'там', 'карельский', 'население', 'это', 'абсолютный', 'противоположность', 'тот', 'карельский', 'население', 'который', 'я', 'всегда', 'встречать', 'это', 'человек', 'который', 'готовый', 'по', 'любой', 'повод', 'поругаться', 'всегда', 'в', 'любой', 'время', 'день', 'ночь', 'год', 'вообще', 'они', 'главное', 'чтобы', 'был', 'конфликт', 'постоянно']
['у', 'нас', 'сегодня', 'второй', 'команда', 'в', 'эссойло', 'поехать']
['а', 'ну', 'вот']
['и', 'причём', 'они', 'очень', 'сильный', 'но', 'я', 'не', 'сказать', 'что', 'они', 'плохой', 'человек', 'нет', 'просто', 'у', 'них', 'вот', 'такой', 'вот', 'сообщество']
None
['вот', 'темперамент', 'сообщество', 'свой', 'они', 'бесконечно', 'готовый', 'ругаться', 'даже', 'когда', 'уже', 'вообще', 'нет', 'причина', 'они', 'всё', 'равно', 'найти']
['вы', 'именно', 'о', '

['я', 'точно', 'помнить', 'что', 'мы', 'песенка', 'изучать', 'на', 'карельский', 'прям', 'хором', 'их', 'петь', 'это', 'было', 'интересно', 'потом', 'мы', 'загадываться', 'загадка', 'и', 'разгадка', 'были', 'например', 'ну', 'там', 'разный', 'существо', 'который', 'на', 'карельский', 'карельский', 'мифология', 'мы', 'их', 'проходить', 'там', 'вот', 'этот', 'тоже', 'хийся', 'дух', 'лес', 'злой', 'демон', 'водяник', 'всё', 'вот', 'это', 'вот', 'и', 'проходить', 'карельский', 'мифология', 'я', 'помнить', 'что', 'лично', 'мне', 'было', 'интересно', 'поэтому', 'это', 'мне', 'запомниться', 'потом', 'традиция', 'изучать', 'карел', 'ну', 'как', 'бы', 'из', 'разряд', 'основный', 'праздник', 'вот', 'такой', 'вещь', 'и', 'праздновать', 'их', 'я', 'точно', 'помнить', 'что', 'мы', 'ходить', 'и', 'там', 'ты', 'не', 'знать', 'что', 'сегодня', 'праздник', 'но', 'ты', 'говорить', 'что', 'праздник', 'и', 'ты', 'как', 'бы', 'приходить', 'какой', 'нибудь', 'вкусный', 'манный', 'каша', 'с', 'сахар', 'есть'

...

['нет', 'вообще', 'небольшой', 'курс', 'у', 'нас', 'по', 'человек', 'но', 'кафедра', 'уже', 'год', 'приезжать', 'учиться', 'ребята', 'из', 'другой', 'финный', 'угорский', 'республика', 'коми', 'мордва', 'удмуртия', 'потому', 'что', 'это', 'единственный', 'кафедра', 'такой', 'специализировать', 'где', 'можно', 'познакомиться', 'с', 'финный', 'угорский', 'мир']
['рассказать', 'пожалуйста', 'а', 'как', 'вы', 'выбрать', 'работать', 'именно', 'здесь', 'или', 'по', 'распределение']
['после', 'училище', 'я', 'попасть', 'по', 'распределение', 'в', 'ведлозеро', 'работать', 'хотя', 'конечно', 'же', 'как', 'все', 'молодой', 'хотеться', 'в', 'петрозаводск', 'у', 'меня', 'там', 'работа', 'уже', 'была', 'но', 'распределение', 'тогда', 'еще', 'существовать', 'я', 'год', 'проработать', 'в', 'консерватория', 'поступить', 'надо', 'сказать', 'что', 'я', 'ехать', 'в', 'ведлозеро', 'я', 'не', 'хотеть', 'туда', 'ехать', 'и', 'прямо', 'когда', 'я', 'уже', 'уезжать', 'из', 'ведлозеро', 'через', 'год', 'уезжат

['я', 'приехать', 'сюда', 'в', 'год', 'после', 'окончание', 'университет', 'жениться', 'в', 'год', 'жена', 'моя', 'инф', 'она', 'кончить', 'тоже', 'петрозаводский', 'университет', 'она', 'местный', 'и', 'я', 'сам', 'беломорский', 'и', 'поэтому', 'я', 'попроситься', 'а', 'она', 'на', 'год', 'ранний', 'меня', 'закончить', 'отправить', 'по', 'место', 'жена', 'меня', 'отправить', 'сюда', 'сначала', 'механик', 'задаром', 'потом', 'инженер', 'год', 'потом', 'год', 'директор', 'в', 'ом', 'год', 'совхоз', 'перестать', 'существовать', 'все', 'расход', 'банк', 'обанкротиться', 'в', 'россия', 'знать', 'не', 'все', 'но', 'больший', 'часть', 'и', 'после', 'банкротство', 'госзавхоз', 'я', 'организовать', 'участок', 'при', 'администрация', 'село', 'жкх', 'и', 'стать', 'сначала', 'мастер', 'потом', 'директор', 'жкх', 'я', 'проработать', 'до', 'го', 'год', 'в', 'ом', 'год', 'случиться', 'инсульт', 'но', 'я', 'уже', 'был', 'на', 'пенсия', 'и', 'вот', 'с', 'го', 'год', 'я', 'заниматься', 'общественный', 

['это', 'не', 'положительный', 'это', 'надо', 'здоровый', 'образ', 'жизнь', 'вести', 'ты', 'говорить', 'молодёжь', 'чтоб', 'пропагандировать', 'изначально', 'а', 'не', 'тогда', 'когда', 'уже', 'но', 'жизнь', 'научить', 'к', 'что', 'то', 'видимо', 'уже', 'но', 'это', 'уже', 'случиться', 'когда', 'на', 'пенсия', 'выйти', 'освоить', 'я', 'к', 'тот', 'стать', 'говорить', 'что', 'аз', 'и', 'прочее', 'и', 'прочее', 'эту', 'техника', 'компьютер', 'потому', 'что', 'я', 'честно', 'не', 'владеть', 'уж', 'что', 'что', 'но', 'вот', 'не', 'хотеть', 'не', 'знать', 'но', 'это', 'же', 'как', 'динозавр', 'туда', 'меня']
['ну', 'конечно', 'если', 'бы', 'я', 'не', 'заниматься', 'компьютер', 'чем', 'я', 'заниматься', 'там', 'рука', 'правый', 'я', 'ж', 'правша', 'перейти', 'на', 'левый', 'на', 'левый', 'рука', 'все', 'работать', 'на', 'компьютер', 'и', 'ручка', 'писать', 'но', 'занятие', 'найти', 'хоть', 'чем', 'то', 'заниматься', 'а', 'так', 'с', 'ум', 'сойти']
['ребёнок', 'говорить', '«наш', 'хакер»']
['

['ну', 'а', 'так', 'мы', 'же', 'вообще', 'сначала', 'у', 'нас', 'все', 'начаться', 'с', 'того', 'еще', 'ирина', 'алексеевич', 'в', 'тот', 'год', 'когда', 'первый', 'год', 'когда', 'была', 'глава', 'стать', 'делать', 'энциклопедия', 'наш', 'вешкеленский', 'сельский', 'поселение', 'вот', 'с', 'этого', 'все', 'начаться', 'потом', 'тоже', 'редколлегия', 'тот', 'же', 'редколлегия', 'выбрать', 'стать', 'материал', 'собирать', 'о', 'человек', 'и', 'вот', 'получиться', 'что', 'это', 'разный', 'слово', 'на', 'карельский', 'русский', 'по', 'русски', 'все', 'название', 'деревня', 'ну', 'в', 'общий', 'что', 'касаться', 'что', 'было', 'связать', 'с', 'вешкелица', 'вот', 'с', 'этого', 'словарь', 'все', 'начаться', 'потому', 'что', 'выиграть', 'тогда', 'грант', 'первый', 'наверное', 'и', 'на', 'тысяча', 'вот', 'этот', 'словарь', 'мы', 'напечатать', 'а', 'сколько', 'там', 'экземпляр', 'потом', 'вот', 'это', 'была', 'тоже', 'презентация', 'и', 'пойти', 'а', 'потом', 'уже', 'стать', 'задуматься', 'об', 

['ранний', 'в', 'совхоз', 'была', 'такой', 'очередь', 'на', 'жильё', 'там', 'на', 'новый', 'на', 'вторичный', 'комиссия', 'последний', 'комитет', 'заниматься', 'по', 'мой', 'район', 'а', 'теперь', 'квартира', 'пустовать']
['потому', 'что', 'приватизировать']
['продавать', 'в', 'город', 'приезжать', 'собственность', 'человек', 'дача', 'строить', 'вокруг']
['поэтому', 'если', 'было', 'бы', 'тут', 'производство', 'понятно', 'что', 'человек', 'бы', 'было', 'большой', 'новый', 'производство', 'а', 'сейчас', 'что', 'сейчас', 'есть', 'вот', 'у', 'нас', 'крестьянский', 'фермерский', 'хозяйство', 'на', 'территория', 'звероферма', 'там', 'семья', 'а', 'так', 'конечно', 'производство', 'нету', 'а', 'так', 'обслуживающий', 'сфера', 'весь', 'остаться', 'торговля', 'есть', 'магазин', 'но', 'нет', 'постоянный', 'фельдшер', 'работа', 'вот', 'тоже', 'проблема']
['а', 'откуда', 'фельдшер']
['приезжать', 'скорый', 'и', 'свой', 'и', 'сейчас', 'ходить', 'слава', 'бог', 'есть', 'медсестра', 'вроде', 'как', 

In [85]:
bag

['вот и в общий в ведлозеро мы тоже заехать обязательно но вот сегодня мы у вас',
 'да там с наташа лучше всего с антонов карельский поэтесса и переводчица об этом разговаривать',
 'мхм да мы знакомый даже с ней',
 'ну зара такой хороший языковой активист в этом план а с ассоциация эхо вы общаться',
 'нет а рассказать что это такой за ассоциация',
 'это ассоциация этнокультурный центр карелия там просто работать два специалист который очень серьёзный уровень',
 'ой а поделиться имя их',
 'да алина чубуров и алексей цыкарев цыкарев он вообще',
 'а конечно конечно знать да да',
 'и у них тоже очень много как бы с язык но они отойти так немножко от этой тема но лёша понятно не отойти только алина так уйти в подполье немного но у них этот',
 'о спасибо это мы обратить внимание да но про лёша мы слышать мы знать',
 'он сейчас в карелия если что то есть его можно дёрнуть и пообщаться то есть он вот в этом план конечно такой очень выс ну очень хороший масштаб специалист в том план что он очен

In [86]:
df['text_ready'] = bag

In [87]:
df

,utterance_id,speaker_id,speaker_role,interwiev_id,text,utterance_number,text_ready
0,1,3,interviewer,1,"Вот, и, в общем, в Ведлозеро мы тоже заедем об...",1,вот и в общий в ведлозеро мы тоже заехать обяз...
1,2,1,informant,1,Да. там с Наташей лучше всего с Антоновой [кар...,2,да там с наташа лучше всего с антонов карельск...
2,3,3,interviewer,1,"Мхм. Да, мы знакомы даже с ней.",3,мхм да мы знакомый даже с ней
3,4,1,informant,1,"Ну, Зара такой хороший языковой активист в это...",4,ну зара такой хороший языковой активист в этом...
4,5,3,interviewer,1,"Нет, а расскажите, что это такая за ассоциация?",5,нет а рассказать что это такой за ассоциация
...,...,...,...,...,...,...,...
3373,3374,52,informant,20,"С цепи срывают. Если бы не на цепи, она бы убе...",154,с цепь срывать если бы не на цепь она бы убежа...
3374,3375,51,informant,20,Или из будки вытаскивают.,155,или из будка вытаскивать
3375,3376,52,informant,20,"В последние годы на балкон медведи, тоже видел...",156,в последний год на балкон медведь тоже видеть ...
3376,3377,51,informant,20,"Мы когда строили дачу, у нас была собака, стор...",157,мы когда строить дача у нас была собака сторож...


In [88]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=1000)
vectors = matrix.fit_transform(bag).toarray()

In [43]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [53]:
df

,utterance_id,speaker_id,speaker_role,interwiev_id,text,utterance_number,text_ready
0,1,3,interviewer,1,"Вот, и, в общем, в Ведлозеро мы тоже заедем об...",1,вот и в общий в ведлозеро мы тоже заехать обяз...
1,2,1,informant,1,Да. там с Наташей лучше всего с Антоновой [кар...,2,да там с наташа лучше всего с антонов карельск...
2,3,3,interviewer,1,"Мхм. Да, мы знакомы даже с ней.",3,мхм да мы знакомый даже с ней
3,4,1,informant,1,"Ну, Зара такой хороший языковой активист в это...",4,ну зара такой хороший языковой активист в этом...
4,5,3,interviewer,1,"Нет, а расскажите, что это такая за ассоциация?",5,нет а рассказать что это такой за ассоциация
...,...,...,...,...,...,...,...
3373,3374,52,informant,20,"С цепи срывают. Если бы не на цепи, она бы убе...",154,с цепь срывать если бы не на цепь она бы убежа...
3374,3375,51,informant,20,Или из будки вытаскивают.,155,или из будка вытаскивать
3375,3376,52,informant,20,"В последние годы на балкон медведи, тоже видел...",156,в последний год на балкон медведь тоже видеть ...
3376,3377,51,informant,20,"Мы когда строили дачу, у нас была собака, стор...",157,мы когда строить дача у нас была собака сторож...


In [54]:
topics = []
for i in range(10):
    print(df['text'][i])
    topics.append(input())

Вот, и, в общем, в Ведлозеро мы тоже заедем обязательно, но вот сегодня мы у вас.
a
Да. там с Наташей лучше всего с Антоновой [карельская поэтесса и переводчица] об этом разговаривать. 
a,b
Мхм. Да, мы знакомы даже с ней.
b
Ну, Зара такой хороший языковой активист в этом плане. А с ассоциацией “Эхо” вы общались?
b,c
Нет, а расскажите, что это такая за ассоциация?
c
Это ассоциация этнокультурных центров Карелии. Там просто работают два специалиста, которые очень серьёзного уровня. 
c
Ой, а поделитесь именами их?
a,c
Да, Алина Чубурова и Алексей Цыкарев. Цыкарев он вообще…
a
А, конечно, конечно знаем, да, да.
b
И у них тоже очень много как бы с языком, но они отошли так немножко от этой темы, но Лёша понятно не отошёл, только Алина так ушла в подполье немного. Но у них этот…
b


In [55]:
type(topics)

list

In [89]:
topics = ['contacts','contacts',
 'contacts',
 'language activist, contacts',
 'language activist, contacts',
 'language activist, contacts',
 'contacts',
 'contacts',
 'contacts',
 'contacts',
 'contacts',
 'language activist',
 'language activist, contacts',
 'biography',
 'biography',
 'biography',
 'biography',
 'other',
 'other',
 'biography',
 'biography, ethnicity',
 'biography',
 'biography',
 'biography, language',
 'biography, language, school',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language',
 'kindergarden, language, culture',
 'language, language politics',
 'other',
 'language politics',
 'language politics, language, language institutions',
 'language politics, language, language institutions',
 'language politics, language',
 'migrants, language',
 'migrants',
 'contacts',
 'contacts',
 'contacts',
 'contacts',
 'contacts',
 'language, dialects',
 'language, dialects',
 'language, dialects',
 'language, dialects',
 'language, contacts',
 'language, contacts',
 'language',
 'language, dialects',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions',
 'language, language institutions, dialects',
 'language, language institutions',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'language, biography',
 'biography',
 'biography',
 'biography',
 'biography, language',
 'language',
 'language',
 'language',
 'language']

In [90]:
df["topics"] = np.nan

In [91]:
df['topics'][:100] = topics

<ipython-input-91-a1da90c42ea0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topics'][:100] = topics
C:\Users\nasty\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [92]:
df

,utterance_id,speaker_id,speaker_role,interwiev_id,text,utterance_number,text_ready,topics
0,1,3,interviewer,1,"Вот, и, в общем, в Ведлозеро мы тоже заедем об...",1,вот и в общий в ведлозеро мы тоже заехать обяз...,contacts
1,2,1,informant,1,Да. там с Наташей лучше всего с Антоновой [кар...,2,да там с наташа лучше всего с антонов карельск...,contacts
2,3,3,interviewer,1,"Мхм. Да, мы знакомы даже с ней.",3,мхм да мы знакомый даже с ней,contacts
3,4,1,informant,1,"Ну, Зара такой хороший языковой активист в это...",4,ну зара такой хороший языковой активист в этом...,"language activist, contacts"
4,5,3,interviewer,1,"Нет, а расскажите, что это такая за ассоциация?",5,нет а рассказать что это такой за ассоциация,"language activist, contacts"
...,...,...,...,...,...,...,...,...
3373,3374,52,informant,20,"С цепи срывают. Если бы не на цепи, она бы убе...",154,с цепь срывать если бы не на цепь она бы убежа...,NaN
3374,3375,51,informant,20,Или из будки вытаскивают.,155,или из будка вытаскивать,NaN
3375,3376,52,informant,20,"В последние годы на балкон медведи, тоже видел...",156,в последний год на балкон медведь тоже видеть ...,NaN
3376,3377,51,informant,20,"Мы когда строили дачу, у нас была собака, стор...",157,мы когда строить дача у нас была собака сторож...,NaN


In [93]:
topics_df = (df.topics.str.split('\s*,\s*', expand=True)
   .stack()
   .str.get_dummies()
   .sum(level=0))
topics_df

,biography,contacts,culture,dialects,ethnicity,kindergarden,language,language activist,language institutions,language politics,migrants,other,school
0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,0,0,0,0,1,0,0,0,0,0,0
96,0,0,0,0,0,0,1,0,0,0,0,0,0
97,0,0,0,0,0,0,1,0,0,0,0,0,0
98,0,0,0,0,0,0,1,0,0,0,0,0,0


In [95]:
final = df.join(topics_df)[:99]
final

,utterance_id,speaker_id,speaker_role,interwiev_id,text,utterance_number,text_ready,topics,biography,contacts,...,dialects,ethnicity,kindergarden,language,language activist,language institutions,language politics,migrants,other,school
0,1,3,interviewer,1,"Вот, и, в общем, в Ведлозеро мы тоже заедем об...",1,вот и в общий в ведлозеро мы тоже заехать обяз...,contacts,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,informant,1,Да. там с Наташей лучше всего с Антоновой [кар...,2,да там с наташа лучше всего с антонов карельск...,contacts,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,3,interviewer,1,"Мхм. Да, мы знакомы даже с ней.",3,мхм да мы знакомый даже с ней,contacts,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,informant,1,"Ну, Зара такой хороший языковой активист в это...",4,ну зара такой хороший языковой активист в этом...,"language activist, contacts",0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,3,interviewer,1,"Нет, а расскажите, что это такая за ассоциация?",5,нет а рассказать что это такой за ассоциация,"language activist, contacts",0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,1,informant,1,"А потом уже здесь в военном городке осели, но ...",95,а потом уже здесь в военный городок осесть но ...,biography,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,96,3,interviewer,1,И ему не с кем было вепсский практиковать особ...,96,и ему не с кто было вепсский практиковать особ...,"biography, language",1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
96,97,1,informant,1,"Да нет, вепсов-то вообще мало, совсем.",97,да нет вепс то вообще мало совсем,language,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
97,98,2,informant,1,Их же только у нас небольшой совсем как бы ест...,98,они же только у нас небольшой совсем как бы ес...,language,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
categories = topics_df.columns.tolist()

train, test = train_test_split(final, random_state=42, test_size=0.33, shuffle=True)
X_train = train.text_ready
X_test = test.text_ready
print(X_train.shape)
print(X_test.shape)

(66,)
(33,)


In [117]:
df.text_ready[100:150]

100                      и вы с аня анхимов общаться уже
101                 нет мы вот в шёлтозеро ещё не ездить
102    аня там наверное в петрозаводск сейчас поэтому...
103                                      о хорошо да это
104    аня с удовольствие я думать пообщаться тем бол...
105               о о о это очень здорово мы обязательно
106    я знать ещё один вепка очень интересный это ак...
107                                  тоже в петрозаводск
108                                                     
109                               тоже в петрозаводск да
110                                                     
111                                                     
112                                     ани с ани с кино
113                        да национальный театр карелия
114                                 аха и аня рахимов да
115                                   аня рахимов и саша
116                                         аня ан химов
117                            

**Naive Bayes**

In [131]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing biography
Test accuracy is 0.8484848484848485
... Processing contacts
Test accuracy is 0.7878787878787878
... Processing culture
Test accuracy is 1.0
... Processing dialects
Test accuracy is 0.9696969696969697
... Processing ethnicity
Test accuracy is 1.0
... Processing kindergarden
Test accuracy is 0.8484848484848485
... Processing language
Test accuracy is 0.6060606060606061
... Processing language activist
Test accuracy is 0.9090909090909091
... Processing language institutions
Test accuracy is 0.7575757575757576
... Processing language politics
Test accuracy is 0.9393939393939394
... Processing migrants
Test accuracy is 0.9696969696969697
... Processing other
Test accuracy is 0.9696969696969697
... Processing school
Test accuracy is 1.0


In [127]:
valid_df = df.text_ready[100:300]
valid_df

100                      и вы с аня анхимов общаться уже
101                 нет мы вот в шёлтозеро ещё не ездить
102    аня там наверное в петрозаводск сейчас поэтому...
103                                      о хорошо да это
104    аня с удовольствие я думать пообщаться тем бол...
                             ...                        
295                                      именно из пряжа
296    да и как бы человек кто задумываться ну ладный...
297    я правильно понимать что у вас например фински...
298                                                     
299                                                     
Name: text_ready, Length: 200, dtype: object

In [132]:
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(valid_df)
    print(prediction.sum())

... Processing biography
0.0
... Processing contacts
0.0
... Processing culture
0.0
... Processing dialects
0.0
... Processing ethnicity
0.0
... Processing kindergarden
0.0
... Processing language
200.0
... Processing language activist
0.0
... Processing language institutions
0.0
... Processing language politics
0.0
... Processing migrants
0.0
... Processing other
0.0
... Processing school
0.0


**LinearSVC**

In [138]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing biography
Test accuracy is 0.8787878787878788
... Processing contacts
Test accuracy is 0.7575757575757576
... Processing culture
Test accuracy is 1.0
... Processing dialects
Test accuracy is 0.9696969696969697
... Processing ethnicity
Test accuracy is 1.0
... Processing kindergarden
Test accuracy is 0.8787878787878788
... Processing language
Test accuracy is 0.6363636363636364
... Processing language activist
Test accuracy is 0.9090909090909091
... Processing language institutions
Test accuracy is 0.8181818181818182
... Processing language politics
Test accuracy is 0.9393939393939394
... Processing migrants
Test accuracy is 0.9696969696969697
... Processing other
Test accuracy is 0.9696969696969697
... Processing school
Test accuracy is 1.0


In [139]:
train['language']

89    1.0
5     0.0
45    0.0
69    1.0
35    1.0
     ... 
60    1.0
71    1.0
14    0.0
92    0.0
51    1.0
Name: language, Length: 66, dtype: float64

In [140]:
X_train

89                                     откуда он уехать
5     это ассоциация этнокультурный центр карелия та...
45                                                     
69               ну как бы редакция полностью смениться
35    есть во весь район в этом план ну это чёткий в...
                            ...                        
60                                       ты видеть нрзб
71       нет её вообще по мой сейчас особо не выпускать
14                                       я тут родиться
92    они какой то время в медвежьегорск жить ну они...
51    там в тот край тоже в прошлый год вот бывать а...
Name: text_ready, Length: 66, dtype: object

In [142]:
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(valid_df)
    print(prediction.sum())

... Processing biography
0.0
... Processing contacts
0.0
... Processing culture
0.0
... Processing dialects
0.0
... Processing ethnicity
0.0
... Processing kindergarden
0.0
... Processing language
0.0
... Processing language activist
0.0
... Processing language institutions
0.0
... Processing language politics
0.0
... Processing migrants
0.0
... Processing other
0.0
... Processing school
0.0


**Logistic Regression**

In [114]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing biography
Test accuracy is 0.7878787878787878
... Processing contacts
Test accuracy is 0.7878787878787878
... Processing culture
Test accuracy is 1.0
... Processing dialects
Test accuracy is 0.9696969696969697
... Processing ethnicity
Test accuracy is 1.0
... Processing kindergarden
Test accuracy is 0.8484848484848485
... Processing language


C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Test accuracy is 0.6060606060606061
... Processing language activist
Test accuracy is 0.9090909090909091
... Processing language institutions
Test accuracy is 0.7575757575757576
... Processing language politics
Test accuracy is 0.9393939393939394
... Processing migrants
Test accuracy is 0.9696969696969697
... Processing other
Test accuracy is 0.9696969696969697
... Processing school
Test accuracy is 1.0


C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [144]:
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(valid_df)
    print(prediction.sum())

... Processing biography
0.0
... Processing contacts
0.0
... Processing culture
0.0
... Processing dialects
0.0
... Processing ethnicity
0.0

C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "



... Processing kindergarden
0.0
... Processing language
200.0
... Processing language activist
0.0
... Processing language institutions
0.0
... Processing language politics
0.0
... Processing migrants
0.0
... Processing other
0.0
... Processing school
0.0


C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\nasty\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
